In [1]:
# All Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

In [2]:
from Network_in_Network import *
from data_utility import *


In [3]:
 train_x, train_y, test_x, test_y = prepare_data()

======Loading data======
DataSet aready exist!
Loading ./cifar-10-batches-py/data_batch_1 : 10000.
Loading ./cifar-10-batches-py/data_batch_2 : 10000.
Loading ./cifar-10-batches-py/data_batch_3 : 10000.
Loading ./cifar-10-batches-py/data_batch_4 : 10000.
Loading ./cifar-10-batches-py/data_batch_5 : 10000.
Loading ./cifar-10-batches-py/test_batch : 10000.
Train data: (50000, 32, 32, 3) (50000, 10)
Test data : (10000, 32, 32, 3) (10000, 10)
======Load finished======
======Shuffling data======
======Prepare Finished======


In [4]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

In [5]:
# Note: Linux bash commands start with a "!" inside those "ipython notebook" cells

DATA_PATH = "data/"

DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)


Dataset is now located at: data/UCI HAR Dataset/


In [6]:
TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    # Substract 1 to each output class for friendly 0-based indexing
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

In [7]:
###cs dataset
ph1=128
ph2=9
compression_nodenum=ph1*ph2
obmatrix = 1 * np.random.randn(1152,compression_nodenum) +0
obmatrix=obmatrix.astype("float32")
def cs(input):
    temp=np.zeros((1,1152))
    output=np.zeros((input.shape[0],ph1,ph2))
    for i in range(input.shape[0]):
        temp=input[i].reshape((1,1152))
        output[i]=np.matmul(temp,obmatrix).reshape((ph1,ph2))
    return output

In [8]:
def low_filter(input):
    sig_num=input.shape[0]
    for i in range (sig_num):
        for j in range(input.shape[1]):
            for k in range (input.shape[2]):
                if abs(input[i][j][k])<=0.1:
                    input [i][j][k]=0
    return input

In [9]:
X_train=low_filter(X_train)
X_test=low_filter(X_test)
X_train=cs(X_train)
X_test=cs(X_test)

In [10]:
# Input Data

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 128# Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 1500  # Loop 300 times on the dataset
batch_size = 1500
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2947, 128, 9) (2947, 1) -0.871628266132 13.7933576407
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [11]:
#Utility functions for training:
def LSTM_RNN(_X, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters.
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network.
    # Note, some code of this notebook is inspired from an slightly different
    # RNN architecture used on another dataset, some of the credits goes to
    # "aymericdamien" under the MIT license.

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input])
    # new shape: (n_steps*batch_size, n_input)

    # ReLU activation, thanks to Yu Zhao for adding this improvement here:
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data.

    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index]

    return batch_s


def one_hot(y_, n_classes=n_classes):
    # Function to encode neural one-hot output labels from number indexes
    # e.g.:
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]

    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [12]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs =         extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))
    #print(batch_xs.shape,batch_ys.shape)
    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs,
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)

    # Evaluate network only at some steps for faster training:
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):

        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))

        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy],
            feed_dict={
                x: X_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data

one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost],
    feed_dict={
        x: X_test,
        y: one_hot(y_test)
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))

Training iter #1500:   Batch Loss = 5.289440, Accuracy = 0.20200000703334808
PERFORMANCE ON TEST SET: Batch Loss = 6.3092041015625, Accuracy = 0.31082457304000854
Training iter #30000:   Batch Loss = 3.166356, Accuracy = 0.7326666712760925
PERFORMANCE ON TEST SET: Batch Loss = 3.374556064605713, Accuracy = 0.6868001222610474
Training iter #60000:   Batch Loss = 2.919750, Accuracy = 0.8213333487510681
PERFORMANCE ON TEST SET: Batch Loss = 3.2059788703918457, Accuracy = 0.7271801829338074
Training iter #90000:   Batch Loss = 2.721871, Accuracy = 0.8740000128746033
PERFORMANCE ON TEST SET: Batch Loss = 3.1456758975982666, Accuracy = 0.7251442074775696
Training iter #120000:   Batch Loss = 2.780158, Accuracy = 0.7940000295639038
PERFORMANCE ON TEST SET: Batch Loss = 3.138788938522339, Accuracy = 0.7203935980796814
Training iter #150000:   Batch Loss = 2.520599, Accuracy = 0.8806666731834412
PERFORMANCE ON TEST SET: Batch Loss = 3.002852201461792, Accuracy = 0.749236524105072
Training iter 

Training iter #1500000:   Batch Loss = 0.562166, Accuracy = 0.9346666932106018
PERFORMANCE ON TEST SET: Batch Loss = 1.3314000368118286, Accuracy = 0.7913131713867188
Training iter #1530000:   Batch Loss = 0.488520, Accuracy = 0.9860000014305115
PERFORMANCE ON TEST SET: Batch Loss = 1.238331913948059, Accuracy = 0.7774007320404053
Training iter #1560000:   Batch Loss = 0.491293, Accuracy = 0.972000002861023
PERFORMANCE ON TEST SET: Batch Loss = 1.3583914041519165, Accuracy = 0.7753647565841675
Training iter #1590000:   Batch Loss = 0.544927, Accuracy = 0.9240000247955322
PERFORMANCE ON TEST SET: Batch Loss = 1.2637330293655396, Accuracy = 0.7767220735549927
Training iter #1620000:   Batch Loss = 0.471112, Accuracy = 0.9413333535194397
PERFORMANCE ON TEST SET: Batch Loss = 1.256760835647583, Accuracy = 0.8004750609397888
Training iter #1650000:   Batch Loss = 0.483113, Accuracy = 0.9340000152587891
PERFORMANCE ON TEST SET: Batch Loss = 1.2865512371063232, Accuracy = 0.783847987651825
Tr

Training iter #3000000:   Batch Loss = 0.270237, Accuracy = 0.9353333115577698
PERFORMANCE ON TEST SET: Batch Loss = 0.8850686550140381, Accuracy = 0.7909738421440125
Training iter #3030000:   Batch Loss = 0.215863, Accuracy = 0.972000002861023
PERFORMANCE ON TEST SET: Batch Loss = 0.8569185733795166, Accuracy = 0.8194774389266968
Training iter #3060000:   Batch Loss = 0.232027, Accuracy = 0.9446666836738586
PERFORMANCE ON TEST SET: Batch Loss = 0.9091482162475586, Accuracy = 0.7987784147262573
Training iter #3090000:   Batch Loss = 0.271499, Accuracy = 0.9313333630561829
PERFORMANCE ON TEST SET: Batch Loss = 0.9423527121543884, Accuracy = 0.7899559140205383
Training iter #3120000:   Batch Loss = 0.222412, Accuracy = 0.9546666741371155
PERFORMANCE ON TEST SET: Batch Loss = 0.9045363664627075, Accuracy = 0.80861896276474
Training iter #3150000:   Batch Loss = 0.217627, Accuracy = 0.9513333439826965
PERFORMANCE ON TEST SET: Batch Loss = 0.8837281465530396, Accuracy = 0.8082796335220337
T

PERFORMANCE ON TEST SET: Batch Loss = 0.8754289150238037, Accuracy = 0.8157448172569275
Training iter #4500000:   Batch Loss = 0.189691, Accuracy = 0.9573333263397217
PERFORMANCE ON TEST SET: Batch Loss = 0.860266923904419, Accuracy = 0.8164234757423401
Training iter #4530000:   Batch Loss = 0.159980, Accuracy = 0.9739999771118164
PERFORMANCE ON TEST SET: Batch Loss = 0.8956031799316406, Accuracy = 0.8103155493736267
Training iter #4560000:   Batch Loss = 0.189408, Accuracy = 0.9606666564941406
PERFORMANCE ON TEST SET: Batch Loss = 0.9013463854789734, Accuracy = 0.7865626215934753
Training iter #4590000:   Batch Loss = 0.192985, Accuracy = 0.9386666417121887
PERFORMANCE ON TEST SET: Batch Loss = 0.8830364346504211, Accuracy = 0.7913131713867188
Training iter #4620000:   Batch Loss = 0.173667, Accuracy = 0.9679999947547913
PERFORMANCE ON TEST SET: Batch Loss = 0.9038897752761841, Accuracy = 0.7960637807846069
Training iter #4650000:   Batch Loss = 0.205245, Accuracy = 0.9380000233650208

In [ ]:
# (Inline plots: )
%matplotlib inline

font = {
    'family' : 'Bitstream Vera Sans',
    'weight' : 'bold',
    'size'   : 18
}
matplotlib.rc('font', **font)

width = 12
height = 12
plt.figure(figsize=(width, height))

indep_train_axis = np.array(range(batch_size, (len(train_losses)+1)*batch_size, batch_size))
plt.plot(indep_train_axis, np.array(train_losses),     "b--", label="Train losses")
plt.plot(indep_train_axis, np.array(train_accuracies), "g--", label="Train accuracies")

indep_test_axis = np.append(
    np.array(range(batch_size, len(test_losses)*display_iter, display_iter)[:-1]),
    [training_iters]
)
plt.plot(indep_test_axis, np.array(test_losses),     "b-", label="Test losses")
plt.plot(indep_test_axis, np.array(test_accuracies), "g-", label="Test accuracies")

plt.title("Training session's progress over iterations")
plt.legend(loc='upper right', shadow=True)
plt.ylabel('Training Progress (Loss or Accuracy values)')
plt.xlabel('Training iteration')

plt.show()